In [ ]:
#!/usr/bin/env python
# encoding: utf-8

In [ ]:
import csv
import pandas as pd 
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import datetime
import matplotlib.pyplot as plt
import time
#import numpy as np
from pprint import pprint

import spacy  # For preprocessing
import nltk
import string
from nltk.corpus import stopwords
import preprocessor as p  #pip install tweet-preprocessor
import logging  # Setting up the loggings to monitor gensim
from nltk.stem import WordNetLemmatizer
from string import punctuation as punc

import gensim
import gensim.corpora as corpora
#from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.models as gsm
from gensim.test.utils import datapath

import regex
import emoji #pip install emoji --upgrade
# Internal dependencies
import word_emoji2vec as we2v
#from word_emoji2vec import Word_Emoji2Vec

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torchvision.transforms as transforms
from torch.autograd import Variable

import os
from datetime import datetime
import argparse
import torch.multiprocessing as mp
import torchvision
import torchvision.transforms as transforms
import torch.distributed as dist
from apex.parallel import DistributedDataParallel as DDP 
from apex import amp
#os.environ['CUDA_VISIBLE_DEVICES'] = '1, 2, 3' 
#....to install apex...
#pip install -v --no-cache-dir ./

nlp = spacy.load('en', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed #python -m spacy download en

torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.enabled = False
np.random.seed(1)

# torch.manual_seed(1)
# np.random.seed(1)


In [ ]:
#load network embedding
net_emb = gsm.KeyedVectors.load_word2vec_format('data/userNetworkEmd.emd')

In [ ]:
df = pd.read_csv("data/yoga_user_name_loc_des_mergetweets_yoga_1300_lb.csv") 
df.sample(10)

In [ ]:
##### load train users and split into train and validation #######
import random
with open("data/train.txt", "r") as f:
    data = f.read().split('\n')
random.seed(1)
random.shuffle(data)

train_data = data[:830] #80% train  
#print(train_data, len(train_data)) #830
valid_data = data[830:] #20% validation
#print(valid_data)


In [ ]:
####.........Create dataframe for Train data
name = []
location = []
description = []
text = []
utype = []
umotivation = []
for i in range (0,len(train_data)):
    for j in range (0,df.shape[0]):
        if df.name[j] == train_data[i]:
            name.append(df.name[j])
            location.append(df.location[j])
            description.append(df.description[j])
            text.append(df.text[j])
            utype.append(df.utype[j])
            umotivation.append(df.umotivation[j])
            break


df_train = pd.DataFrame(data= name, columns=['Name'])
df_train['location'] = location
df_train['description'] = description   
df_train['text'] = text
df_train['utype'] = utype  
df_train['umotivation'] = umotivation   


In [ ]:
####.........Create dataframe for validation data
name = []
location = []
description = []
text = []
utype = []
umotivation = []
for i in range (0,len(valid_data)):
    for j in range (0,df.shape[0]):
        if df.name[j] == valid_data[i]:
            name.append(df.name[j])
            location.append(df.location[j])
            description.append(df.description[j])
            text.append(df.text[j])
            utype.append(df.utype[j])
            umotivation.append(df.umotivation[j])
            break


df_valid = pd.DataFrame(data= name, columns=['Name'])
df_valid['location'] = location
df_valid['description'] = description   
df_valid['text'] = text
df_valid['utype'] = utype  
df_valid['umotivation'] = umotivation   


In [ ]:
########.....Load Test data.......
with open("data/test.txt", "r") as f:
    data = f.read().split('\n')
test_data = data[:] 
#print(test_data, len(test_data)) #262

In [ ]:
####.........Create dataframe for Test data
name = []
location = []
description = []
text = []
utype = []
umotivation = []
for i in range (0,len(test_data)):
    for j in range (0,df.shape[0]):
        if df.name[j] == test_data[i]:
            name.append(df.name[j])
            location.append(df.location[j])
            description.append(df.description[j])
            text.append(df.text[j])
            utype.append(df.utype[j])
            umotivation.append(df.umotivation[j])
            break


df_test = pd.DataFrame(data= name, columns=['Name'])
df_test['location'] = location
df_test['description'] = description   
df_test['text'] = text
df_test['utype'] = utype  
df_test['umotivation'] = umotivation   


In [ ]:
###### text preprocessing #########

def remove_string_noise(input_str):
    input_str = re.sub(r"http\S+", "", input_str)
    
    #give special char you want to remove
    #do not put space between chars, and space (" ") is not a special char
    punctuation_noise ="!\"$%&'#()*+,-./:;<=>?@[\]^_`{|}~" #print string.punctuation 
    #number_noise = "0123456789"
    special_noise = ""

    #all_noise = punctuation_noise + number_noise + special_noise
    all_noise = punctuation_noise + special_noise

    for c in all_noise:
        if c in input_str:
            input_str = input_str.replace(c, " ")#replace with space
    fresh_str = ' '.join(input_str.split())
    return fresh_str

def clean_tweets(tweet):


    wordnet_lemmatizer = WordNetLemmatizer()
#     sw = set(stopwords.words('english'))
#     operators = set(('no', 'not', 'nor', 'none'))
#     stop_words = set(sw) - operators
#     stop_words.update([ 'amp', 'rt'])  ###as we are using set so we used .update....otherwise .extends
    
#     tweet = re.sub(r':', '', tweet)
#     tweet = re.sub(r'‚Ä¶', '', tweet)
#     tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    word_tokens = nltk.word_tokenize(tweet)
    filtered_tweet = []
    for w in word_tokens:
        #if w not in stop_words and w not in emoticons:
        #if w not in stop_words:
        filtered_tweet.append(w)
    lemmatized_tweet = []
    for word in filtered_tweet:
        lemmatized_tweet.append(wordnet_lemmatizer.lemmatize(word, pos="v"))

    #print (sentences)
    #return ' '.join(filtered_tweet)
    return ' '.join(lemmatized_tweet)
    #return ' '.join(sentences)
    
def pre_processing_tweets (df):
    #p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY, p.OPT.MENTION, p.OPT.RESERVED)
    p.set_options(p.OPT.EMOJI, p.OPT.SMILEY, p.OPT.MENTION, p.OPT.RESERVED)
    
    clean_text = []
    #for i in range (0, df.shape[0]):
        #clean_text.append(p.clean(str(df['text'][i])))#python 3

    for i in df:
        clean_text.append(p.clean(str(i)))#python 3
    #print(clean_text) #all ok
    fresh_text1 = []
    for i in range (0, df.shape[0]):
        #fresh_text1.append(remove_string_noise(clean_text[i].encode('ascii', 'ignore').decode("utf-8"))) #can remove other emojis and no \UF..
        fresh_text1.append(remove_string_noise(clean_text[i]))
    #print(fresh_text1) #all ok
    
    #Call clean_tweet method-2 for extra preprocessing
    filtered_tweet = []
    for i in range (0, len(fresh_text1)):
        filtered_tweet.append(clean_tweets(fresh_text1[i].lower()))
    return filtered_tweet
    

In [ ]:
sentences_train_loc = pre_processing_tweets(df_train.location)
sentences_valid_loc = pre_processing_tweets(df_valid.location)
sentences_test_loc = pre_processing_tweets(df_test.location)

In [ ]:
sentences_train_des = pre_processing_tweets(df_train.description)
sentences_valid_des = pre_processing_tweets(df_valid.description)
sentences_test_des = pre_processing_tweets(df_test.description)

In [ ]:
sentences_train_twt = pre_processing_tweets(df_train.text)
sentences_valid_twt = pre_processing_tweets(df_valid.text)
sentences_test_twt = pre_processing_tweets(df_test.text)

In [ ]:
# Get the lists of sentences and their labels.

sentences_train_loc = np.array(sentences_train_loc)
sentences_train_des = np.array(sentences_train_des)
sentences_train_twt = np.array(sentences_train_twt)
labels_train = torch.tensor(df_train.umotivation.values)
#print(type(sentences_train_loc))

sentences_valid_loc = np.array(sentences_valid_loc)
sentences_valid_des = np.array(sentences_valid_des)
sentences_valid_twt = np.array(sentences_valid_twt)
labels_valid = torch.tensor(df_valid.umotivation.values)
#print(type(sentences_valid_loc))

sentences_test_loc = np.array(sentences_test_loc)
sentences_test_des = np.array(sentences_test_des)
sentences_test_twt = np.array(sentences_test_twt)
labels_test = torch.tensor(df_test.umotivation.values)
#print(type(sentences_test_loc))

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
#The first four features are in tokenizer.encode, but I’m using tokenizer.encode_plus to get the fifth item (attention masks).

# Tokenize all of the sentences and map the tokens to thier word IDs.
def token_id(sentences):
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in sentences:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = bert_tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 160,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            truncation=True, #explicitely truncate examples to max length. #my add
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])
    #print(len(input_ids), type(input_ids), input_ids[0], input_ids[0].size() ) #torch.Size([1, 160])
    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    #labels = torch.tensor(labels)

#     # Print sentence 0, now as a list of IDs.
#     print('Original: ', sentences_train[0])
#     print('Token IDs:', input_ids_train[0])
    return input_ids, attention_masks


In [ ]:
input_ids_train_loc, attention_masks_train_loc = token_id(sentences_train_loc )
input_ids_valid_loc, attention_masks_valid_loc = token_id(sentences_valid_loc )
input_ids_test_loc, attention_masks_test_loc = token_id(sentences_test_loc)

In [ ]:
input_ids_train_des, attention_masks_train_des = token_id(sentences_train_des )
input_ids_valid_des, attention_masks_valid_des = token_id(sentences_valid_des )
input_ids_test_des, attention_masks_test_des = token_id(sentences_test_des)

In [ ]:
from transformers import LongformerModel, LongformerTokenizer, LongformerConfig
longformer_tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096', do_lower_case=True)


In [ ]:
#The first four features are in tokenizer.encode, but I’m using tokenizer.encode_plus to get the fifth item (attention masks).

# Tokenize all of the sentences and map the tokens to thier word IDs.
def token_id_twt(sentences):
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in sentences:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = longformer_tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 4000,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            truncation=True, #explicitely truncate examples to max length. #my add
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
        #print(type(input_ids[0]), input_ids[0], input_ids[0].size())
        #sys.exit()

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    #labels = torch.tensor(labels)

#     # Print sentence 0, now as a list of IDs.
#     print('Original: ', sentences_train[0])
#     print('Token IDs:', input_ids_train[0])
    return input_ids, attention_masks


In [ ]:
input_ids_train_twt, attention_masks_train_twt = token_id_twt(sentences_train_twt)
input_ids_valid_twt, attention_masks_valid_twt = token_id_twt(sentences_valid_twt )
input_ids_test_twt, attention_masks_test_twt = token_id_twt(sentences_test_twt)

In [ ]:
input_ids_train_twt.size()

In [ ]:
#####prepare user network data for neural net#########
def nn_input_network(train_data,df):
    training_data =[]
    for i in range (0, len(train_data)):
    #for i in range (0, 10):
        for j in range (0,df.shape[0]):
            if (train_data[i] == df.name[j]):
                #print(train_data[i]) #print username
                utype =  [int(df.utype[j])]
                umotivation = [int(float(df.umotivation[j]))]
                #print ("net_emb[train_data[i]] : ", net_emb[train_data[i]], type(net_emb[train_data[i]]), torch.Tensor(net_emb[train_data[i]]), type(torch.Tensor(net_emb[train_data[i]])))
                #count = 0
                if(train_data[i] not in net_emb ):
                    net_emb[train_data[i]] = np.zeros(300) #For users not appearing in the mention network, we set their network embedding vectors as 0.
                    #count = count + 1
                #print(count)
                #print(net_emb[train_data[i]]) #ok
                ####.....convert ndarray to torch.tensor........
                net_emb_tensor = torch.Tensor(net_emb[train_data[i]])
                #print(net_emb_tensor) #ok
                training_data.append(net_emb_tensor.unsqueeze(0))
                break
    return training_data


In [ ]:
#####prepare training data for neural net #########
training_data_net =  nn_input_network(train_data,df)
#####prepare validation data for neural net #########
validation_data_net =  nn_input_network(valid_data,df)
#####prepare Testing data for neural net #########
testing_data_net =  nn_input_network(test_data,df)

In [ ]:
#print(len(training_data_net), type(training_data_net), training_data_net[0], training_data_net[0].size() ) #torch.Size([1, 160])

In [ ]:
# Convert the network lists into tensors.
training_data_net = torch.cat(training_data_net, dim=0)
validation_data_net = torch.cat(validation_data_net, dim=0)
testing_data_net = torch.cat(testing_data_net, dim=0)

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids_train_des, input_ids_train_loc, training_data_net, input_ids_train_twt, labels_train)
val_dataset = TensorDataset(input_ids_valid_des, input_ids_valid_loc, validation_data_net, input_ids_valid_twt, labels_valid)
test_dataset = TensorDataset(input_ids_test_des, input_ids_test_loc, testing_data_net, input_ids_test_twt, labels_test)

In [ ]:
# import torch
# # torch.cuda.empty_cache() 
# # If there's a GPU available...
# if torch.cuda.is_available():    

#     # Tell PyTorch to use the GPU.    
#     #device = torch.device("cuda") # select the zeroth GPU with this line: gpu = 0
#     #device = torch.cuda.set_device(1)  #wrong provide device = None  
#     device = torch.device(3) #(use cuda device 1) for gpu = 1
#     print('There are %d GPU(s) available.' % torch.cuda.device_count())

#     print('We will use the GPU:', torch.cuda.get_device_name())

# # If not...
# else:
#     print('No GPU available, using the CPU instead.')
#     device = torch.device("cpu")

device = torch.device("cpu") # uncomment for cpu use



In [ ]:
# #print ('Current cuda device ', torch.cuda.current_device())
# print ('Current cuda device ', device)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )



In [ ]:
from transformers import BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
long_roberta = LongformerModel.from_pretrained('allenai/longformer-base-4096', gradient_checkpointing=True)

In [ ]:
#####....Longformer encoding for Tweets
from torchnlp.nn import Attention #pip imstall pytorch-nlp
class ROBERTALSTMSentiment(nn.Module):
    def __init__(self,
                 long_roberta,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.long_roberta = long_roberta
        
        embedding_dim = long_roberta.config.to_dict()['hidden_size'] #768
              
        self.rnn = nn.LSTM(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        
        
        self.dropout = nn.Dropout(dropout)
        self.attn_fc = Attention(hidden_dim * 2 if bidirectional else hidden_dim) #attention layer from torchnlp
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        #self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, hidden_dim)
        #self.fc1 = nn.Linear(150, output_dim) 
        #self.fc1 = nn.Linear(hidden_dim, output_dim) 
        #self.fc2 = nn.Linear(50, output_dim)

    def forward(self, text):
        
        #text = [batch size, sent len]
        #print("text", text.size()) #torch.Size([32, 50])       
        with torch.no_grad():
            embedded = self.long_roberta(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        #print("embedded", type(embedded), embedded.size()) #torch.Size([32, 50, 768])
        #embedded = self.dropout(embedded) #add dropout...don't use it...dropout layer after an pre-trained embedding, that the weights are not learnt during training
        lstm_out, (hidden, c_n) = self.rnn(embedded) #for lstm
        
        #hidden = [n layers * n directions, batch size, hid dim]
        #print("hidden before", type(hidden), hidden.size()) #torch.Size([4, 32, 256])
        if self.rnn.bidirectional: #add dropout
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        
#         if self.rnn.bidirectional: #No dropout
#             hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
#         else:
#             hidden = hidden[-1,:,:]
            
        #lstm_out = self.dropout(lstm_out) #add dropout
    
        #hidden = [batch size, hid dim]
        #print("hidden after", type(hidden), hidden.size()) #torch.Size([32, 512]) cz of bi-direction = 2 * 256 = 512
        #attn_out = self.attention(lstm_out, hidden)
#         query = hidden = [batch size, output length, dimensions]
#         context = lstm_out = [batch size, query length, dimensions]
#        input for attention attention(query, context)
#       output for attention tuple with output and weights
        attn_out = self.attn_fc(hidden.unsqueeze(1), lstm_out)
        #print(type(attn_out[0]), attn_out[0].size(), type(attn_out[1]), attn_out[1].size()) #output = torch.Size([32, 1, 512]), weight = torch.Size([32, 1, 160])
        #print("attn_out", attn_out.size())
        #output = self.out(hidden)
        
        #output = self.out(attn_out[0].squeeze(1)) #only tweet
        #return output
        return attn_out[0].squeeze(1)

In [ ]:
class BERTLSTMSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size'] #768
              
        self.rnn = nn.LSTM(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        
        
        self.dropout = nn.Dropout(dropout)
        self.attn_fc = Attention(hidden_dim * 2 if bidirectional else hidden_dim) #attention layer from torchnlp
        #self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        #self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, hidden_dim)
        #self.fc1 = nn.Linear(150, output_dim) 
        #self.fc1 = nn.Linear(hidden_dim, output_dim) 
        #self.fc2 = nn.Linear(50, output_dim)
    
    def attention(self, rnn_out, state):
        #print("rnn_out", rnn_out.size())        
        #print("state", state.size()) 
        state = state.unsqueeze(2)
        #print("state", state.size())
        # (batch, seq_len, cell_size) * (batch, cell_size, 1) = (batch, seq_len, 1)
        weights = torch.bmm(rnn_out, state)
        #print("weights", weights.size()) #torch.Size([150, 13, 1])
        #weights = torch.nn.functional.softmax(weights.squeeze(2)).unsqueeze(2)
        weights = F.log_softmax(weights.squeeze(2),dim = 1).unsqueeze(2)
         #F.log_softmax(x, dim = 1)
        #print("weights2 :", weights.size()) #torch.Size([150, 13, 1])
        # (batch, cell_size, seq_len) * (batch, seq_len, 1) = (batch, cell_size, 1)
        return torch.bmm(torch.transpose(rnn_out, 1, 2), weights).squeeze(2)
    # end method attention

    
    def forward(self, text):
        
        #text = [batch size, sent len]
        #print("text", text.size()) #torch.Size([32, 50])       
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        #print("embedded", type(embedded), embedded.size()) #torch.Size([32, 50, 768])
        #embedded = self.dropout(embedded) #add dropout...don't use it...dropout layer after an pre-trained embedding, that the weights are not learnt during training
        lstm_out, (hidden, c_n) = self.rnn(embedded) #for lstm
        
        #hidden = [n layers * n directions, batch size, hid dim]
        #print("hidden before", type(hidden), hidden.size()) #torch.Size([4, 32, 256])
        if self.rnn.bidirectional: #add dropout
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        
#         query = hidden = [batch size, output length, dimensions]
#         context = lstm_out = [batch size, query length, dimensions]
#         input for attention attention(query, context)
#         output for attention tuple with output and weights

        #attn_out = self.attn_fc(hidden.unsqueeze(1), lstm_out)
        #print(type(attn_out[0]), attn_out[0].size(), type(attn_out[1]), attn_out[1].size()) #output = torch.Size([32, 1, 512]), weight = torch.Size([32, 1, 160])
        #print("attn_out", attn_out[0].size())
        #output = self.out(hidden)
        
        #output = self.out(attn_out[0].squeeze(1)) #original
        #output = F.relu(output)
        #print("output", type(output), output.size())
        
        #output = self.fc1(output)
        #output = self.fc2(output)
        #output = [batch size, out dim]
        #print("output", type(output), output.size()) #torch.Size([32, 3])
        #return output ##original
        #print(hidden.size()) #torch.Size([32, 600])
        return hidden
        #return attn_out[0].squeeze(1)
        #attn_out = self.attention(lstm_out, hidden)
        #return attn_out

In [ ]:
########## Create Model 
class NetworkMLP(nn.Module):
    def __init__(self):
        super(NetworkMLP, self).__init__() 
        self.fc1 = nn.Linear(300, 150)
        
    def forward(self, X):
        #print(X.size()) # torch.Size([32, 300])
        z1 = self.fc1(X)
        #print('z1', z1, z1.size()) # torch.Size([32, 150])
        #h1 = F.relu(z1) 
        return z1 
        #return h1

In [ ]:
class JointModel(nn.Module):
    #def __init__(self):
    def __init__(self,
                 bert,long_roberta,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
    
        #super(JointModel, self).__init__()
        super().__init__()
        self.model_des = BERTLSTMSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)
        self.model_loc = BERTLSTMSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)
        self.model_net = NetworkMLP()
        self.model_twt = ROBERTALSTMSentiment(long_roberta,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)
        self.fc1 = nn.Linear((hidden_dim * 6 +150) if bidirectional else (hidden_dim * 3 +150), 600)
        self.fc2 = nn.Linear(600, output_dim) #original
#         self.fc2 = nn.Linear(900, 200) #200
#         self.fc3 = nn.Linear(200, output_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x_d, x_l, x_n, x_t): 
        prediction_des = self.model_des(x_d)
        #print(prediction_des, prediction_des.size()) 
        prediction_loc = self.model_loc(x_l)
        #print(prediction_loc, prediction_loc.size()) 
        prediction_net = self.model_net(x_n)
        #print(prediction_net, prediction_net.size()) 
        prediction_twt = self.model_twt(x_t)
        concat_pred = torch.cat((prediction_des, prediction_loc, prediction_net, prediction_twt), 1) #concat with dim= 1
        #print(concat_pred, concat_pred.size()) #torch.Size([1, 6])
        output = self.fc1(concat_pred)
        output = self.dropout(output) # add dropout
        output = self.fc2(F.relu(output)) #original
#         output = self.dropout(output)#add dropout
#         output = self.fc3(F.relu(output))
        return output
    

In [ ]:
# HIDDEN_DIM = 256
# OUTPUT_DIM = 3
# N_LAYERS = 2
# BIDIRECTIONAL = True
# DROPOUT = 0.25

#HIDDEN_DIM = 256 #original
HIDDEN_DIM = 300 #good
OUTPUT_DIM = 3
N_LAYERS = 2 #original good
#N_LAYERS = 3
BIDIRECTIONAL = True
#DROPOUT = 0.25 #original
DROPOUT = 0.5 #good

model = JointModel(bert, long_roberta, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL,DROPOUT)

# model = BERTLSTMSentiment(bert,
#                          HIDDEN_DIM,
#                          OUTPUT_DIM,
#                          N_LAYERS,
#                          BIDIRECTIONAL,
#                          DROPOUT)

# model = ROBERTALSTMSentiment(long_roberta,
#                          HIDDEN_DIM,
#                          OUTPUT_DIM,
#                          N_LAYERS,
#                          BIDIRECTIONAL,
#                          DROPOUT)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
###In order to freeze paramers (not train them) we need to set their requires_grad attribute to False. To do this, we simply loop through all of the named_parameters in our model and if they're a part of the bert transformer model, we set requires_grad = False.

for name, param in model.named_parameters():    
    #print(name)
    if name.startswith('bert') or name.startswith('long_roberta'):
        param.requires_grad = False

In [ ]:

import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
#criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()

In [ ]:
model = model.to(device) 
criterion = criterion.to(device) 

# # If we use DataParallel
# #  It distributes training into multiple GPUs in a single machine

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if torch.cuda.device_count() > 1:
#     print("Let's use", torch.cuda.device_count(), "GPUs!")
#     # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
#     model = nn.DataParallel(model)

# model.to(device)
# criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    #rounded_preds = torch.round(torch.sigmoid(preds))
    #rounded_preds = torch.round(preds)
    correct = (preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    
    from sklearn.metrics import f1_score
    macro_f1 = f1_score(y.to("cpu"), preds.to("cpu"), average='macro')
    #print("macro_f1", macro_f1)

    return acc, macro_f1

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_macro = 0
    model.train()
    for step, batch in enumerate(iterator):    
    #for batch in iterator:
        #print("batch", batch)
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
#         b_input_ids = batch[0].to(device)
#         b_input_mask = batch[1].to(device)
#         b_labels = batch[2].to(device)

        optimizer.zero_grad()
        des = batch[0].to(device)
        loc = batch[1].to(device)
        net = batch[2].to(device)
        twt = batch[3].to(device)
        label = batch[4].to(device)
        #print("label", label, type(label),label.size()) #torch.Size([32])
        #label = label.unsqueeze(1)
        #print("label", label, type(label),label.size())
        #predictions = model(batch.text).squeeze(1)
        #predictions = model(text)
        predictions = model(des, loc, net, twt)
        #pred = model(text)
        #print("pred", pred.size()) #torch.Size([32, 3])
        #predictions = torch.argmax(pred, dim = 1)
        #print("predictions", predictions, predictions.size()) #torch.Size([32, 3])
        #loss = criterion(predictions, batch.label)
        loss = criterion(predictions, label)
        #print("loss", loss)
        #acc = binary_accuracy(predictions, batch.label)
        acc, macro_f1 = binary_accuracy(torch.argmax(predictions, dim = 1), label)
        #print("acc", acc)
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_macro += macro_f1.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_macro / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_macro = 0
    model.eval()
    
    with torch.no_grad():
    
        #for batch in iterator:
        for step, batch in enumerate(iterator):    
            des = batch[0].to(device)
            loc = batch[1].to(device)
            net = batch[2].to(device)
            twt = batch[3].to(device)
            label = batch[4].to(device)

            #predictions = model(batch.text).squeeze(1)
            predictions = model(des, loc, net, twt)
            #loss = criterion(predictions, batch.label)
            loss = criterion(predictions, label)
            #acc = binary_accuracy(predictions, batch.label)
            acc, macro_f1 = binary_accuracy(torch.argmax(predictions, dim = 1), label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_macro += macro_f1.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_macro / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:

N_EPOCHS = 10

best_valid_loss = float('inf')
per_epoch_train_loss = []
per_epoch_val_loss = []
per_epoch_train_f1 = []
per_epoch_val_f1 = []
per_epoch_train_acc = []
per_epoch_val_acc = []
for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc, train_f1 = train(model, train_dataloader, optimizer, criterion)
    valid_loss, valid_acc, valid_f1 = evaluate(model, validation_dataloader, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    per_epoch_train_loss.append(train_loss)
    per_epoch_val_loss.append(valid_loss)
    per_epoch_train_f1.append(train_f1)
    per_epoch_val_f1.append(valid_f1)
    per_epoch_train_acc.append(train_acc)
    per_epoch_val_acc.append(valid_acc)
    
    if valid_loss <= best_valid_loss:
        best_valid_loss = valid_loss
        print("best model saved in epoch :", epoch+1 )
    torch.save(model.state_dict(), 'data/bert_des+loc+net+twt/dlnt_umotivation'+str(epoch+1)+'.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f} | Train macro-avg-f1: {train_f1*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f} |  Val. macro-avg-f1: {valid_f1*100:.2f}%')
    
    
    

In [ ]:
def save_plots(train_losses, val_losses, train_f1, val_f1, train_accs, val_accs):
    """Plot

        Plot two figures: loss vs. epoch and accuracy vs. epoch
    """
    n = len(train_losses)
    xs = np.arange(1,n+1,1)

    # plot train and val losses
    fig, ax = plt.subplots()
    ax.plot(xs, train_losses, '--', linewidth=2, label='train loss')
    ax.plot(xs, val_losses, '-', linewidth=2, label='validation loss')
    #ax.set_xlim(0, 10)
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.legend(loc='upper right')
    plt.savefig('data/bert_plot/loss_Bert_DLNT_umotivation.png')

    # plot train and val f1-score
    #plt.clf()
    fig, ax = plt.subplots()
    ax.plot(xs, train_f1, '--', linewidth=2, label='train')
    ax.plot(xs, val_f1, '-', linewidth=2, label='validation')
    
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Macro-avg F1")
    ax.legend(loc='lower right')
    plt.savefig('data/bert_plot/f1_Bert_DLNT_umotivation.png')
    
    # plot train and val accuracy
    #plt.clf()
    fig, ax = plt.subplots()
    ax.plot(xs, train_accs, '--', linewidth=2, label='train')
    ax.plot(xs, val_accs, '-', linewidth=2, label='validation')
    #ax.set_xlim(0, 10)
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Accuracy")
    ax.legend(loc='lower right')
    plt.savefig('data/bert_plot/acc_Bert_DLNT_umotivation.png')
save_plots(per_epoch_train_loss, per_epoch_val_loss, per_epoch_train_f1, per_epoch_val_f1, per_epoch_train_acc, per_epoch_val_acc)

In [ ]:
# For Test the order doesn't matter, so we'll just read them sequentially.
test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
# model.load_state_dict(torch.load('data/bert_des+loc+net/dln_model_utype1.pt'))

# test_loss, test_acc, test_f1 = evaluate(model, test_dataloader, criterion)

# print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f} | Test macro-avg-f1: {test_f1*100:.2f}%')

In [ ]:
#epoch 3: Test Loss: 0.595 | Test Acc: 77.55 | Test macro-avg-f1: 68.37%


In [ ]:
for epoch in range(0,10):
#     model = JointModel(bert, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL,DROPOUT)
#     model = model.to(device)
    model.load_state_dict(torch.load('data/bert_des+loc+net+twt/dlnt_umotivation'+str(epoch+1)+'.pt')) 
    test_loss, test_acc, test_f1 = evaluate(model, test_dataloader, criterion)

    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f} | Test macro-avg-f1: {test_f1*100:.2f}%')